# Week 5 Account Data Cleaning Activity

**Data Set:** Data_ Cleaning_Activity_Data.csv
**Data Cleaning Tool:** Python

***

Sebastian Espinoza



Task 1: Import all necessary libraries & get familiar w/ data.

For this project pandas is the data cleaning tool of choice, along with plotly, which will be used for creating interactive visualizations of the data.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as py

# Shows plots on HTML file offline mode
py.init_notebook_mode(connected=True)

### Task 2: Import data

The data is read in as a comma separated values file.

In [2]:
acc_df = pd.read_csv('Data_Cleaning_Activity_Data.csv')

acc_df.head()

,account_ID,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,collection_agency_code,date_turnover,agency_code,agency,amount_owed,amount_paid,account_closed
0,668164_35205805,32310-7340,23,2983,5.0,2.25,46,0,384.0,2.0,33.0,11/9/2017,33.0,Agency_B,860.27,2.09,11/28/2017
1,668164_35205805,32310-4570,23,2983,NaN,2.25,46,0,NaN,NaN,NaN,NaN,33.0,Agency_B,NaN,199.47,11/28/2017
2,668164_35205805,32310-4995,23,2983,NaN,2.25,46,0,NaN,NaN,NaN,NaN,33.0,Agency_B,NaN,9.84,11/28/2017
3,668164_35205805,32310-5661,23,2983,NaN,2.25,46,0,NaN,NaN,NaN,NaN,33.0,Agency_B,NaN,2.09,11/28/2017
4,867201_369287913,32010-1535,23,202,1.0,0.25,3,0,0.0,0.0,75.0,7/3/2017,75.0,Agency_A,146.40,1.20,7/24/2017


### Task 3: Check data types of each variable

Using the info() function, the different data types stored within each column can be seen. The data type "object" signifies a string.

In [3]:
acc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107339 entries, 0 to 107338
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   account_ID                    107339 non-null  object 
 1   zip_code                      107339 non-null  object 
 2   state_code                    107339 non-null  int64  
 3   days_account_open             107339 non-null  int64  
 4   kids_count                    36934 non-null   float64
 5   average_credit_history_score  107339 non-null  float64
 6   on_time_payments              107339 non-null  int64  
 7   max_status                    107339 non-null  int64  
 8   credit_score                  31175 non-null   float64
 9   returned_checks               31175 non-null   float64
 10  collection_agency_code        47468 non-null   float64
 11  date_turnover                 47468 non-null   object 
 12  agency_code                   77047 non-null

### Task 4: Fill down columns

The fill_down function fills down data for kids_count, credit_score, returned_checks, collection_agency_code, and amount_owed. 

The date_turnover column is filled in by the fill_down_dates function. Accounts that only have only NA rows for date_turnover are remained unchanged. Accounts with one non-NA value are filled down using said value. Accounts with more than one non-NA value are first filled using the last valid value. Then each date_turnover and account_closed pair is checked to ensure that the date_turnover occurs before account_closed date. If date_turnover does not satisfy this condition, date_turnover is set to the earliest date for that account_ID group.

_Note: Dates were turned into timestamp objects for ease of use._

<br>
<br>


This code snippet was tried, but it only returned values from the account_ID column.

`xdf['date_turnover'] = np.where((xdf['account_closed'] - xdf['date_turnover']).dt.days > 0, np.NaN, min_date)`


In [4]:
def fill_down_dates(xdf):

    # Check if there are zero non-NA rows
    if xdf['date_turnover'].count() == 0:
        print(xdf['date_turnover'])
        pass
    
    # Check if there is only one non-NA row
    elif xdf['date_turnover'].count() == 1:
        xdf['date_turnover'] = xdf['date_turnover'].ffill()

    
    else:
        
        # Fill NA rows with last date_turnover value
        xdf['date_turnover'].fillna(method='ffill', inplace=True)
            
        # Iterate through rows
        # Keep turnover days less than account_closed, replace ones greater with the minimum account_closed date
        for row in xdf.itertuples():
            
            # Current date_turnover row value
            fval = xdf.at[row.Index, 'date_turnover']

            # Change rows that have account_closed dates after date_turnover
            if (xdf.at[row.Index,'account_closed'] - xdf.at[row.Index, 'date_turnover']).days < 0:

                fval = xdf['date_turnover'].min()

            xdf.at[row.Index, 'date_turnover'] = fval


    return xdf['date_turnover']

In [5]:
def fill_down(df):

    # Change dates to datetime objects
    df['account_closed'] = pd.to_datetime(df['account_closed'])
    df['date_turnover'] = pd.to_datetime(df['date_turnover'])

    cols = ['kids_count', 'credit_score', 'returned_checks',
                           'collection_agency_code', 'amount_owed']

    # Group by account ID and forward fill data
    df[cols] = df.groupby(['account_ID'], sort=False, as_index=False)[cols].ffill()

    # Fill down dates
    dfcol = df.groupby(['account_ID'], sort=False).apply(fill_down_dates)
    df['date_turnover'] = dfcol.reset_index(level=0, drop=True)

    return df

acc_df = fill_down(acc_df)

acc_df.head()

,account_ID,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,collection_agency_code,date_turnover,agency_code,agency,amount_owed,amount_paid,account_closed
0,668164_35205805,32310-7340,23,2983,5.0,2.25,46,0,384.0,2.0,33.0,2017-11-09,33.0,Agency_B,860.27,2.09,2017-11-28
1,668164_35205805,32310-4570,23,2983,5.0,2.25,46,0,384.0,2.0,33.0,2017-11-09,33.0,Agency_B,860.27,199.47,2017-11-28
2,668164_35205805,32310-4995,23,2983,5.0,2.25,46,0,384.0,2.0,33.0,2017-11-09,33.0,Agency_B,860.27,9.84,2017-11-28
3,668164_35205805,32310-5661,23,2983,5.0,2.25,46,0,384.0,2.0,33.0,2017-11-09,33.0,Agency_B,860.27,2.09,2017-11-28
4,867201_369287913,32010-1535,23,202,1.0,0.25,3,0,0.0,0.0,75.0,2017-07-03,75.0,Agency_A,146.40,1.20,2017-07-24


### Task 5: Remove Conflicting columns

The data dictionary states that column collection_agency_code contains the same values as agency_code. Since these two are not the same collection_agency_code was dropped from the data frame object.

In [6]:
# Task 5: Conflicting Columns
# Check if collection_agency_code indeed conflicts w/ agency_code
def remove_col_ag_code(df):

    # Check if the two columns are similar
    check_df = np.where(df['collection_agency_code'] == df['agency_code'],
             df['agency_code'], 'no match')

    # If not similar remove the column
    if 'no match' in check_df:
        df = df.drop(['collection_agency_code'], axis=1)

    return df

acc_df = remove_col_ag_code(acc_df)

acc_df.head()

,account_ID,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,date_turnover,agency_code,agency,amount_owed,amount_paid,account_closed
0,668164_35205805,32310-7340,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,33.0,Agency_B,860.27,2.09,2017-11-28
1,668164_35205805,32310-4570,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,33.0,Agency_B,860.27,199.47,2017-11-28
2,668164_35205805,32310-4995,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,33.0,Agency_B,860.27,9.84,2017-11-28
3,668164_35205805,32310-5661,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,33.0,Agency_B,860.27,2.09,2017-11-28
4,867201_369287913,32010-1535,23,202,1.0,0.25,3,0,0.0,0.0,2017-07-03,75.0,Agency_A,146.40,1.20,2017-07-24


#### Task 6: Remove variables not needed for data set

There are variables which are not necessary for this analysis. After creating a dictionary from all of the agency and agency_code combinations it seems that multiple agencies use more than one code. Since we are only interested in the agencies themselves and not synonymous codes, we can drop the agency code column.

In [7]:
# Task 6: Remove a variable that is not needed in the data set
# Since we already have the agency code we don't need the agency tab
def remove_agency(df):

    # Make a dictionary of agency codes and agency
    agency_key = dict(zip(df.agency_code, df.agency))
    agency_key_out = {}


    
    for key in agency_key:
        
        # Remove all nan from dict and print out the codes w/ agency name
        if type(agency_key[key]) is str:
            
            # Add keys to new dictionary w/ lists as values
            if agency_key[key] in agency_key_out:
                agency_key_out[agency_key[key]].append(key)

            else:
                agency_key_out.update({agency_key[key]: [key]})
                

    print(agency_key_out)
    # Drop the agency column
    df = df.drop(['agency_code'], axis=1)

    return df

acc_df = remove_agency(acc_df)

acc_df.head()

{'Agency_B': [33.0, 123.0, 39.0, 42.0], 'Agency_A': [75.0, 78.0], 'Agency_C': [93.0, 96.0], 'Agency_D': [144.0, 147.0], 'Agency_E': [297.0], 'Agency_F': [60.0], 'Bankrupt': [15.0], 'No_Agency_Assigned': [0.0], 'Agency_G': [6.0], 'Agency_J': [192.0, 183.0, 174.0, 180.0, 189.0], 'Deceased': [18.0], 'Agency_H': [108.0, 105.0], 'Disputed_Bill': [21.0], 'Agency_I': [270.0], 'Agency_K': [285.0]}


,account_ID,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,date_turnover,agency,amount_owed,amount_paid,account_closed
0,668164_35205805,32310-7340,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,Agency_B,860.27,2.09,2017-11-28
1,668164_35205805,32310-4570,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,Agency_B,860.27,199.47,2017-11-28
2,668164_35205805,32310-4995,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,Agency_B,860.27,9.84,2017-11-28
3,668164_35205805,32310-5661,23,2983,5.0,2.25,46,0,384.0,2.0,2017-11-09,Agency_B,860.27,2.09,2017-11-28
4,867201_369287913,32010-1535,23,202,1.0,0.25,3,0,0.0,0.0,2017-07-03,Agency_A,146.40,1.20,2017-07-24


### Task 7: Aggregate data

We want to aggregate the data so there aren't multiple rows for the same account_ID and data. Both date_turnover and account_closed dates (along with account_ID) were used to group the data. After the data was grouped, the total amount_paid, mode of kids_count, and last zip_count were calculated. The new aggregated columns were then merged with the original data frame and duplicate columns were dropped.

In [8]:
# Task 7: Aggregate data
def aggregate_data(df):

    # Apply aggregate functions to data
    agg_dict = {'amount_paid': ['sum'], 'kids_count': [pd.Series.mode],
                'zip_code': 'last'}
    df_agg = df.groupby(['account_ID', 'date_turnover', 'account_closed'], sort=False, as_index=False).agg(agg_dict)

    # Rename aggregate data frame columns
    df_agg.columns = ['account_ID', 'date_turnover', 'account_closed', 'amount_paid_total', 'kids_count_mode', 'zip_code']

    # Merge aggregate data w/ dropped account IDs
    merge_cols = ['account_ID', 'date_turnover', 'account_closed']
    df = df.drop(['zip_code', 'amount_paid'], axis=1)

    # Left merge aggregated data with the original dataframe
    x_merge = pd.merge(df_agg, df, how='left', left_on=merge_cols, right_on=merge_cols)

    # Drop duplicate rows
    x_merge = x_merge.drop_duplicates(merge_cols)

    return x_merge

acc_df = aggregate_data(acc_df)

acc_df.head()

,account_ID,date_turnover,account_closed,amount_paid_total,kids_count_mode,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,agency,amount_owed
0,668164_35205805,2017-11-09,2017-11-28,213.49,5.0,32310-5661,23,2983,5.0,2.2500,46,0,384.0,2.0,Agency_B,860.27
4,867201_369287913,2017-07-03,2017-07-24,91.00,1.0,32010-5406,23,202,1.0,0.2500,3,0,0.0,0.0,Agency_A,146.40
8,867201_369287913,2017-07-03,2017-09-08,55.41,1.0,32010-4320,23,202,1.0,0.2500,3,0,0.0,0.0,Agency_A,146.40
12,1459829_859925060,2015-10-20,2016-02-18,91.01,2.0,31010-7022,23,1092,2.0,2.6667,0,4,NaN,NaN,Agency_B,296.51
16,1459829_859925060,2015-10-20,2015-11-03,69.10,2.0,31010-9206,23,1092,2.0,2.6667,0,4,NaN,NaN,Agency_B,296.51


### Task 8: Create bins for different pay periods (i.e. 0-30 days, 31-60 days, etc...)

A new "time_to_pay" column was created which stores strings for each account instance indicating the time difference between account_closed and date_turnover.

Accounts with NaN values after the difference had been calculated, were assigned -1 so they would not be considered in the creation of the day bins.

In [9]:
# Task 8:

# Create a column with integer data on how long it took client to pay
def calc_day_difference(df):

    # Get the difference between account closed and date turnover
    df['time_to_pay'] = df['account_closed'] - df['date_turnover']

    # Mark NaNs as -1
    df['time_to_pay'] = df['time_to_pay'].fillna('-1')
    df['account_closed'] = df['account_closed'].fillna('-1')

    # Change time_to_pay column to string
    df['time_to_pay'] = df['time_to_pay'].astype(str)

    # Change days to int
    df['time_to_pay'] = df['time_to_pay'].str.replace('\sdays.*', '').astype(int)

    return df

# Create column with string identifiers for each interval
def create_day_bins(df):

    conditions = [(df['time_to_pay'] >= 0) & (df['time_to_pay'] <= 30),
                  (df['time_to_pay'] >= 31) & (df['time_to_pay'] <= 60),
                  (df['time_to_pay'] >= 61) & (df['time_to_pay'] <= 90),
                  df['time_to_pay'] >= 90]
    intervals = ['0-30 days', '31-60 days', '61-90 days', '90+ days']

    df['pay_time_intervals'] = np.select(conditions, intervals, default='No time interval')

    return df

acc_df = calc_day_difference(acc_df)
acc_df = create_day_bins(acc_df)

acc_df.head()

,account_ID,date_turnover,account_closed,amount_paid_total,kids_count_mode,zip_code,state_code,days_account_open,kids_count,average_credit_history_score,on_time_payments,max_status,credit_score,returned_checks,agency,amount_owed,time_to_pay,pay_time_intervals
0,668164_35205805,2017-11-09,2017-11-28,213.49,5.0,32310-5661,23,2983,5.0,2.2500,46,0,384.0,2.0,Agency_B,860.27,19,0-30 days
4,867201_369287913,2017-07-03,2017-07-24,91.00,1.0,32010-5406,23,202,1.0,0.2500,3,0,0.0,0.0,Agency_A,146.40,21,0-30 days
8,867201_369287913,2017-07-03,2017-09-08,55.41,1.0,32010-4320,23,202,1.0,0.2500,3,0,0.0,0.0,Agency_A,146.40,67,61-90 days
12,1459829_859925060,2015-10-20,2016-02-18,91.01,2.0,31010-7022,23,1092,2.0,2.6667,0,4,NaN,NaN,Agency_B,296.51,121,90+ days
16,1459829_859925060,2015-10-20,2015-11-03,69.10,2.0,31010-9206,23,1092,2.0,2.6667,0,4,NaN,NaN,Agency_B,296.51,14,0-30 days


### Task 9: Create a histogram showing the number of accounts with a specific number of kids


In [10]:
# Task 9 Create histogram of number of accounts with # of kids
fig_9 = px.histogram(acc_df, x='kids_count_mode', title='Number of Kids for Account Holders',
                  labels = {'kids_count_mode': 'Number of Kids'})

fig_9.show()
#py.iplot(fig_9, filename='fig-9-bar')

### Task 10: Create two grouped bar charts. One should show the total amount of money the top 4 agencies collected over the four time intervals in task eight. The second one should show the total number of unique accounts those top 4 agencies collected.

First, the data frame was grouped by the agency column and then the sum of amount_paid_total was calculated for all unique accounts. Then the top 4 agencies were found using this data. 

The original data frame was then grouped again by agency and pay_time_intervals, and the sum for amount_paid_total was calculated for these groups. Then all of the rows which did not have agencies in the top 4 were filtered out.
This data frame was used for creating the first plot.

A similar approach was used for the data frame for the second plot. The original data frame was again grouped by agency and pay_time_intervals. The size of each group was then calculated (assuming each row is a unique account and date combination) and was used as the total number of unique accounts. The data frame was again filtered to only include data from the top 4 agencies.

In [11]:
def top_4_agencies(df):
    
    # Plot 1 Data
    # Get the sum for the total paid amount for each agency
    groupby_agency = df.groupby(['agency'], sort=False, as_index=False)
    
    # Get the sum of amount_paid_total for unique accounts for each agency
    amount_paid_agency = groupby_agency['amount_paid_total'].sum()
    
    # Sort grouped accounts by amount_paid_total asc
    amount_paid_agency = amount_paid_agency.sort_values(by=['amount_paid_total'], ascending=False)
    
    # Get top 4 agencies
    top4 = amount_paid_agency['agency'].tolist()[:4]
    
    # Group by agency and time invervals
    groupby_agency_time = df.groupby(['agency', 'pay_time_intervals'])
    
    # Total amount paid for each agency for each time interval
    total_paid_per_time = groupby_agency_time['amount_paid_total'].sum().reset_index()
    
    # Only get rows that have the top 4 agencies
    target_rows = total_paid_per_time.agency.isin(top4)
    filtered_total_paid_per_time = total_paid_per_time[target_rows]
    
    # Plot 2 Data
    # Gets the total number of unique accounts (unique ID and date combination)
    total_accounts = groupby_agency_time.size().reset_index()
    total_accounts = total_accounts.rename(columns= {0: 'total_accounts'})
    
    # Only get rows that have the top 4 agencies
    target_rows = total_accounts.agency.isin(top4)
    filtered_total_accounts = total_accounts[target_rows]
    
    
    return filtered_total_paid_per_time, filtered_total_accounts

agency_paid, agency_accounts = top_4_agencies(acc_df)


fig10 = px.bar(agency_paid, x='agency', y='amount_paid_total', color='pay_time_intervals', barmode='group',
               title='Total Money Collected from Top 4 Agencies', 
               labels={'agency': 'Agencies', 'amount_paid_total': 'Amount Paid ($)',
                       'pay_time_intervals': 'Pay Time Intervals'})
fig10.show()

fig10_part2 = px.bar(agency_accounts, x='agency', y='total_accounts', color='pay_time_intervals', barmode='group',
                    title='Total Paying Accounts from Top 4 Agencies',
                    labels={'agency': 'Agencies', 'total_accounts': 'Number of Accounts',
                       'pay_time_intervals': 'Pay Time Intervals'})
fig10_part2.show()



### Download the cleaned data frame as .csv

In [12]:
acc_df.to_csv('cleaned_accounts_data.csv')